# Data Preparation

In [1]:
#import libraries

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

#smote data rebalancing
from imblearn.over_sampling import SMOTE

#normalization
from sklearn import preprocessing

#classification
from sklearn.model_selection import train_test_split
from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show

import warnings
warnings.filterwarnings('ignore')

# Loading Dataset

In [2]:
# https://archive-beta.ics.uci.edu/ml/datasets/cervical+cancer+risk+factors
"""
The dataset was collected at 'Hospital Universitario de Caracas' in Caracas, Venezuela.
The dataset comprises demographic information, habits, and historic medical records of 858 patients.
Several patients decided not to answer some of the questions because of privacy concerns (missing values).
"""

rf = pd.read_csv('risk_factors_cervical_cancer.csv',encoding='utf8')
#rf.info()

# Covert data to usable datatypes

In [3]:
# replace missing values with NaN
rf = rf.replace('?',np.nan)

# covert everything to float64, some classes will be converted to bool once missing values are taken care of
for label, col in rf.iteritems():
    rf[label] = pd.to_numeric(col, errors='coerce')

#TODO: some things are still int after this and object types get transformed to bool automatically. But NaN values
#are not falsly converted to True so that should be okay.

# Examine data

## Check missing values per column

In [4]:
percent_missing = rf.isnull().sum() * 100 / len(rf)
missing_value_df = pd.DataFrame({'column_name': rf.columns,
                                 'percent_missing': percent_missing})
missing_value_df

,column_name,percent_missing
Age,Age,0.000000
Number of sexual partners,Number of sexual partners,3.030303
First sexual intercourse,First sexual intercourse,0.815851
Num of pregnancies,Num of pregnancies,6.526807
Smokes,Smokes,1.515152
Smokes (years),Smokes (years),1.515152
Smokes (packs/year),Smokes (packs/year),1.515152
Hormonal Contraceptives,Hormonal Contraceptives,12.587413
Hormonal Contraceptives (years),Hormonal Contraceptives (years),12.587413
IUD,IUD,13.636364


In [5]:
# remove columns with more than 20% missing
selection_columns = missing_value_df.loc[missing_value_df['percent_missing'] >= 20].iloc[:, 0]

for col in selection_columns:
    rf = rf.drop([col], axis=1)

## Check missing values per row

In [6]:
missing_count = []
for idx in range(len(rf)):
    missing = rf.loc[[idx]].isna().sum().sum()
    missing_count.append(missing)

removed = 0
for idx, val in enumerate(missing_count):
    # Remove all rows where more than 15% of the data are missing
    # If 20% is chosen, 106 patients are excluded
    if val >= len(rf.columns)*0.15:
        rf = rf.drop(idx)
        removed+=1
print("{} number of rows were removed".format(removed))

106 number of rows were removed


## Investigate class imbalances

In [7]:
# get number of patients per group
rf.groupby('Biopsy').count()

,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,...,STDs:Hepatitis B,STDs:HPV,STDs: Number of diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Hinselmann,Schiller,Citology
Biopsy,,,,,,,,,,,,,,,,,,,,,
0,699,686,693,658,691,691,691,687,687,686,...,699,699,699,699,699,699,699,699,699,699
1,53,52,53,47,52,52,52,53,53,51,...,53,53,53,53,53,53,53,53,53,53


## Correlations between variables

In [ ]:
# Plot non-bool parameters
sns.pairplot(rf[['Age','Number of sexual partners','Num of pregnancies','IUD (years)',
                 'Hormonal Contraceptives (years)', 'STDs (number)', 'Smokes (years)']])

## Investigate STDs (number) vs STDs: Number of diagnosis

In [ ]:
# check if STDs (number) and STDs: Number of diagnosis are the same
STD_comparison = np.where((rf['STDs (number)']==rf['STDs: Number of diagnosis']), True, False)

# check manual calculation with STDs(number) if STDS (number) and STDs: Number of diagnosis not the same
if not STD_comparison.all():
    # create a new column with the sum of all STDs
    rf['STDs (number manual)'] = (rf['STDs:condylomatosis'] + rf['STDs:cervical condylomatosis'] +
        rf['STDs:vaginal condylomatosis'] + rf['STDs:vulvo-perineal condylomatosis'] +
        rf['STDs:syphilis'] + rf['STDs:pelvic inflammatory disease'] +
        rf['STDs:genital herpes'] + rf['STDs:molluscum contagiosum'] +
        rf['STDs:AIDS'] + rf['STDs:HIV'] +
        rf['STDs:Hepatitis B'] + rf['STDs:HPV'])
    
    # if they're the same drop the new column again
    manual_comp = np.where((rf['STDs (number)']==rf['STDs (number manual)']), True, False)
    if manual_comp.all():
        rf = rf.drop(['STDs (number manual)'], axis=1)

In [ ]:
# STDs: Number of diagnosis is also not the same as just STDs
STD_comparison = np.where((rf['STDs']==rf['STDs: Number of diagnosis']), True, False).all() #=False

# is STDs: Number of diagnosis a encoded value?
minimum = rf['STDs: Number of diagnosis'].min() #=0
maximum = rf['STDs: Number of diagnosis'].max() #=3
# unlikely that it's the diagnosis code if the max is 3

# plot in histogram
ax = rf.hist(column=["STDs: Number of diagnosis"], bins=maximum)

## Data normalization

In [ ]:
#Decision to not use normalization to keep it interpretable
"""column_names  = rf.columns.values.tolist()

#normalize all columns to 0 to 1
values = rf.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
values_scaled = min_max_scaler.fit_transform(values)
rf_norm = pd.DataFrame(values_scaled)

# rename the columns again
rf_norm.columns = column_names"""

# Pivot table

In [ ]:
group = 'Dx:Cancer'
column_ = 'Dx:HPV'
agg_function = np.mean 

pivot_sum=pd.pivot_table(rf,index=[group],values=[column_],aggfunc=agg_function)
pivot_sum=pivot_sum.reset_index()
pivot_sum

# Splitting the dataset into training and testing sets

In [ ]:
# using oversampled data
# split dataset in features and target variable
feature_cols = rf.columns[1:len(rf.columns)-4]
X = rf[feature_cols] # Features
y = rf.Biopsy # Target variable

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1) # 80% training and 20% test

# Implement splitting for cross-validation here later

## Data imputation
Following the methods described in:
Razali, Nazim & Mostafa, Salama & Mustapha, Aida & Abd Wahab, Mohd Helmy & Ibrahim, Nurul. (2020). Risk Factors of Cervical Cancer using Classification in Data Mining. Journal of Physics: Conference Series. 1529. 022102. 10.1088/1742-6596/1529/2/022102. 

"Missing values for attribute that have integer data type were filled using the sample mean while boolean
were filled using the sample mode."

In [ ]:
# imputation on X_train and X_test
# minority class is not missing any data, known from analysis before

# imputation method 1: subsitute mean/mode of column for missing values -> X_<train/test>_mean_imputation
# imputation method 2: use unique values that don't/can't otherwise exist in the df. Chosen value: -99

X_train_mean_imputation = X_train.copy()
X_test_mean_imputation = X_test.copy()

X_train_unique_imputation = X_train.copy()
X_test_unique_imputation = X_test.copy()

# store columns with specific data type
bool_columns = ['Smokes','Hormonal Contraceptives', 'IUD', 'STDs',
                'STDs:condylomatosis', 'STDs:cervical condylomatosis',
                'STDs:vaginal condylomatosis', 'STDs:vulvo-perineal condylomatosis',
                'STDs:syphilis', 'STDs:pelvic inflammatory disease', 
                'STDs:genital herpes', 'STDs:molluscum contagiosum',
                'STDs:AIDS', 'STDs:HIV', 'STDs:Hepatitis B', 'STDs:HPV',
                'Dx:Cancer', 'Dx:CIN', 'Dx:HPV', 'Dx']

# replace NaN with mode for columns with dtype bool
for label, col in X_train_mean_imputation.iteritems():
    if label in bool_columns:
        columns_mode = col.mode()
        # method 1
        X_train_mean_imputation[label] = col.fillna(columns_mode[0])
        X_test_mean_imputation[label] = col.fillna(columns_mode[0])

        #convert column to bool while we're at it
        X_train_mean_imputation[label] = X_train_mean_imputation[label].astype('bool')
        X_test_mean_imputation[label] = X_test_mean_imputation[label].astype('bool')
    
float_columns = X_train_mean_imputation.select_dtypes(include=['float64']).columns

# replace NaN with mean for columns with dtype float
for col in float_columns:
    columns_mean = X_train_mean_imputation[col].mean()
    X_train_mean_imputation[col] = X_train_mean_imputation[col].fillna(columns_mean)
    X_test_mean_imputation[col] = X_test_mean_imputation[col].fillna(columns_mean)
    
# method 2:
X_train_unique_imputation = X_train_unique_imputation.fillna(-99)
X_test_unique_imputation = X_test_unique_imputation.fillna(-99)

In [ ]:
# Check minority class imbalance
y_train.value_counts()

# Oversampling of X_train, y_train

## Implement Oversampling

In [ ]:
training_data = pd.concat([X_train_unique_imputation, y_train], axis=1) # change here which imputation method to use

# locate all columns where Biopsy is 1
minority_class = training_data.loc[training_data['Biopsy'] == 1]

print("Minority class count before oversampling: \n{}\n".format(training_data['Biopsy'].value_counts()))

# oversample with factor 12.2
minority_class = minority_class.sample(frac=12.2, replace=True, random_state=1)

# concat dataframes
frames = [training_data, minority_class]
training_oversampled = pd.concat(frames)

print("Minority class count after oversampling: \n{}".format(training_oversampled['Biopsy'].value_counts()))

# split into X and y again
X_train_oversampled = training_oversampled.iloc[:,:-1]
y_train_oversampled = training_oversampled.iloc[:,-1:]

## SMOTE oversampling - training data

In [ ]:
# oversampling using synthetic minority oversampling technique (SMOTE)
# see: Chawla N V, Bowyer K W, Hall L O and Kegelmeyer W P 2002 Journal of Artificial Intelligence Research 16 321-357

# for reproducibility purposes
seed = 100
# SMOTE number of neighbors
k = 1

sm = SMOTE(sampling_strategy='auto', k_neighbors=k, random_state=seed)
X_train_smote, y_train_smote = sm.fit_resample(X_train_unique_imputation, y_train)

print("Minority class count after SMOTE oversampling: \n{}".format(y_train_smote.value_counts()))

# Classification

## Explainable Boosting Classfier

In [ ]:
# fit an ebm
ebm = ExplainableBoostingClassifier()
ebm.fit(X_train_oversampled, y_train_oversampled)

In [ ]:
# understand the model
ebm_global = ebm.explain_global()
show(ebm_global)

In [ ]:
# understand individual predictions
ebm_local = ebm.explain_local(X_test_mean_imputation, y_test)
show(ebm_local)

In [ ]:
# Return the mean accuracy on the given test data and labels
acc = ebm.score(X_test_mean_imputation, y_test)
print("The accuracy of the model using mean oversampling is: {}".format(acc))

In [ ]:
# fitting a ebm to the data where smote oversampling was used

# using smote data

# fit an ebm
ebm_smote = ExplainableBoostingClassifier()
ebm_smote.fit(X_train_smote, y_train_smote)

# understand the model
#ebm_smote_global = ebm_smote.explain_global()
#show(ebm_smote_global)

# Return the mean accuracy on the given test data and labels
acc_smote = ebm_smote.score(X_test_mean_imputation, y_test)
print("The accuracy of the model is using smote oversampling is: {}".format(acc_smote))